In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras import optimizers
import numpy as np
import time

# models
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
# 分類するクラス
classes = ['normal','tumor'] 
nb_classes = len(classes)

# 画像サイズ、batch数、epoch数
img_width, img_height = 200, 200
batch_size = 4
nb_epoch = 200

In [3]:
train_data_dir      = 'dataset/train'
validation_data_dir = 'dataset/validation'

nb_train_samples      = 2000
nb_validation_samples = 600

result_dir = 'dataset/results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [4]:
def model_maker():

    input_tensor = Input(shape=(img_width, img_height, 3))
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

    x = vgg16.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    
    return Model(inputs=vgg16.input, outputs=predictions)


In [5]:
def image_generator():
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        zoom_range=0.4,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='reflect')

    validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    return (train_generator, validation_generator)

In [6]:
# モデル作成
model = model_maker()

# 最後の6層以外の重みをfreeze
for layer in model.layers[:len(model.layers) - 7]:
    layer.trainable = False
        
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [ ]:
from keras.callbacks import ModelCheckpoint #重み保存の際、epochを通して最良の重みを保存するため

# モデル重みデータの保存 (monitor: 監視する値, save_best_only=Trueの場合，監視しているデータの最良モデルが上書きされない)
# 各epochで最良の重みが保存される
checkpointer = ModelCheckpoint('ex1/finetuning3.h5', monitor='val_acc', verbose=1, save_best_only=True)    

In [ ]:
# 多クラス分類を指定
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.001),
                  metrics=['accuracy'])

# 画像のジェネレータ生成
train_generator, validation_generator = image_generator()
 
start = time.time()
# Fine-tuning
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples/batch_size,
    validation_steps = nb_validation_samples/batch_size,
    nb_epoch = nb_epoch,
    validation_data = validation_generator,
    callbacks=[checkpointer])

model_json_str = model.save("ex1/model3.h5")

process_time = (time.time() - start) / 60
print(u'学習終了。かかった時間は', process_time, u'分です。')

Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., callbacks=[<keras.ca..., validation_data=<keras.pre..., steps_per_epoch=500.0, epochs=200, validation_steps=150.0)`


Epoch 1/200
500/500 [==============================] - 64s 128ms/step - loss: 0.6524 - acc: 0.6685 - val_loss: 0.2771 - val_acc: 0.8983
Epoch 2/200
500/500 [==============================] - 63s 126ms/step - loss: 0.3205 - acc: 0.8660 - val_loss: 0.2152 - val_acc: 0.9117
Epoch 3/200
500/500 [==============================] - 63s 127ms/step - loss: 0.1913 - acc: 0.9370 - val_loss: 0.2116 - val_acc: 0.9417
Epoch 4/200
500/500 [==============================] - 63s 125ms/step - loss: 0.1660 - acc: 0.9420 - val_loss: 0.3058 - val_acc: 0.8900
Epoch 5/200
500/500 [==============================] - 63s 127ms/step - loss: 0.1345 - acc: 0.9565 - val_loss: 0.2163 - val_acc: 0.9417
Epoch 6/200
500/500 [==============================] - 63s 126ms/step - loss: 0.1328 - acc: 0.9635 - val_loss: 0.2892 - val_acc: 0.9067
Epoch 7/200
500/500 [==============================] - 63s 126ms/step - loss: 0.1144 - acc: 0.9610 - val_loss: 0.2429 - val_acc: 0.9117
Epoch 8/200
500/500 [===========================

In [ ]:
model_json_str = model.save("ex1/model3.h5")


In [ ]:
import matplotlib.pyplot as plt
loss = hist.history['loss']
val_loss = hist.history['val_loss']

# lossのグラフ
plt.plot(range(nb_epoch), loss, marker='.', label='loss')
plt.plot(range(nb_epoch), val_loss, marker='.', label='val_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
acc = hist.history['acc']
val_acc = hist.history['val_acc']

# accuracyのグラフ
plt.plot(range(nb_epoch), acc, marker='.', label='acc')
plt.plot(range(nb_epoch), val_acc, marker='.', label='val_acc')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
#Plotlyを使用してプロットしてみる。

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

##データの元を作る,複数書きたい場合はこれを繰り返す
trace  = go.Scatter(y = hist.history['val_acc'],
                    x=np.arange(1,201) ,
                    name = 'val_acc',
                    mode = 'lines')
trace1 = go.Scatter(y = hist.history['val_loss'],
                    x=np.arange(1,201) ,
                    name = 'val_loss',
                    mode = 'lines')

data = [trace,trace1]


##データのレイアウトを決める
layout = go.Layout(
    title='Fine-Tuning cancer or normal',
    legend={"x":0.8, "y":0.5},#legendそのままにしたいならshowlegend = True

    xaxis={"title":"epoch"},#軸の最大値最小値を決めたいならここに"range": [最小値,最大値]のように記入
    yaxis={"title":"accuracy"}
)

##データをプロット
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,show_link = False)#グラフ表示のExport to plotly を消すにはここにshow_link=Falseのように記入

### スコア

$0.9466667$

`InceptionV2`など別のモデルを試してみたが、ハイパーパラメータの調整がうまくいかず`VGG16`より高スコアを出せなかったため、`VGG16`を使って精度向上を図ることにした。

課題で与えられた元々のモデルでもすでに9割以上の精度が出ているが、１エポック目でほぼ収束しているように見えたため、FC層を３層に増やすとともにオプティマイザーの学習率を小さくしてより細かに探索を行うようにした。  
また、オプティマイザーは`Adam`に変更した。

これにより、`50`エポック目かけて徐々に学習データに対する正答率が上がるようになった。


バリデーションデータに対するスコアは3エポック目に対するものが最高となった。
一方でこの時学習データに対するスコアは`0.6557`しかなく、エポックを重ねるごとに学習データに対する`loss`も`accuracy`も改善していったがバリデーションデータに対するスコアの改善は見られなかった。

ただし、バリデーションデータに対するスコアはほぼ`0.93`を超えており、大きな過学習を起こしているわけではないため、比較的良いモデルになったのではないかと考える。